In [0]:
!pip install transformers

     |████████████████████████████████| 368kB 4.9MB/s 
     |████████████████████████████████| 1.0MB 47.9MB/s 
     |████████████████████████████████| 860kB 54.7MB/s 
     |████████████████████████████████| 645kB 49.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=10ed617d7b2797792cf856247bd97d156acb1fb8db7e375d50201f32ee67fe90
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses


In [0]:
!pip install -U sentence-transformers

     |████████████████████████████████| 51kB 2.5MB/s 
     |████████████████████████████████| 163kB 13.2MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.2.3-cp36-none-any.whl size=57413 sha256=9d00e000f827647c2f4d82ca31094a2dd0efe9c0fdcb7a2e920e0251c6149268
  Stored in directory: /root/.cache/pip/wheels/20/89/88/95d8a3d4034bb77f52a6f66efdbfc623240221183dc0001c31
Successfully built sentence-transformers


In [0]:
"""
This examples trains BERT for the STSbenchmark from scratch. It generates sentence embeddings
that can be compared using cosine-similarity to measure the similarity.
"""
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, losses, models
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import STSDataReader
import logging
from datetime import datetime


#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
#### /print debug information to stdout

In [0]:
# Read the dataset
train_batch_size = 16
num_epochs = 4
model_save_path = 'output/training_n2c2_sts_bert-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
# parameters for STSDataReader -> dataset_folder, s1_col_idx=5, s2_col_idx=6, score_col_idx=4, delimiter="\t", quoting=csv.QUOTE_NONE, normalize_scores=True, min_score=0, max_score=5
sts_reader = STSDataReader('', s1_col_idx=0, s2_col_idx=1, score_col_idx=2, normalize_scores=True)

# Use BERT for mapping tokens to embeddings
word_embedding_model = models.BERT('bert-base-uncased')

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=True,
                               pooling_mode_max_tokens=True)

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

2019-11-28 11:43:46 - https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json not found in cache, downloading to /tmp/tmp36vj2b2l


100%|██████████| 313/313 [00:00<00:00, 125316.64B/s]


2019-11-28 11:43:47 - copying /tmp/tmp36vj2b2l to cache at /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
2019-11-28 11:43:47 - creating metadata file for /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
2019-11-28 11:43:47 - removing temp file /tmp/tmp36vj2b2l
2019-11-28 11:43:47 - loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
2019-11-28 11:43:47 - Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,


100%|██████████| 440473133/440473133 [00:11<00:00, 38139167.05B/s]


2019-11-28 11:43:59 - copying /tmp/tmpgm1aa9gf to cache at /root/.cache/torch/pytorch_transformers/aa1ef1aede4482d0dbcd4d52baad8ae300e60902e88fcb0bebdec09afd232066.36ca03ab34a1a5d5fa7bc3d03d55c4fa650fed07220e2eeebc06ce58d0e9a157
2019-11-28 11:44:00 - creating metadata file for /root/.cache/torch/pytorch_transformers/aa1ef1aede4482d0dbcd4d52baad8ae300e60902e88fcb0bebdec09afd232066.36ca03ab34a1a5d5fa7bc3d03d55c4fa650fed07220e2eeebc06ce58d0e9a157
2019-11-28 11:44:00 - removing temp file /tmp/tmpgm1aa9gf
2019-11-28 11:44:00 - loading weights file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-pytorch_model.bin from cache at /root/.cache/torch/pytorch_transformers/aa1ef1aede4482d0dbcd4d52baad8ae300e60902e88fcb0bebdec09afd232066.36ca03ab34a1a5d5fa7bc3d03d55c4fa650fed07220e2eeebc06ce58d0e9a157
2019-11-28 11:44:03 - https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt not found in cache, downloading to /tmp/tmpzph6okle


100%|██████████| 231508/231508 [00:00<00:00, 1147710.74B/s]


2019-11-28 11:44:04 - copying /tmp/tmpzph6okle to cache at /root/.cache/torch/pytorch_transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
2019-11-28 11:44:04 - creating metadata file for /root/.cache/torch/pytorch_transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
2019-11-28 11:44:04 - removing temp file /tmp/tmpzph6okle
2019-11-28 11:44:04 - loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.cache/torch/pytorch_transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
2019-11-28 11:44:04 - Use pytorch device: cpu


In [0]:
#sts_reader = STSDataReader('', s1_col_idx=0, s2_col_idx=1, score_col_idx=2, normalize_scores=True)

# Convert the dataset to a DataLoader ready for training
logging.info("Read STSbenchmark train dataset")
train_data = SentencesDataset(sts_reader.get_examples('clinicalSTS2019.train_train60.csv'), model)
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model)


logging.info("Read STSbenchmark dev dataset")
dev_data = SentencesDataset(examples=sts_reader.get_examples('clinicalSTS2019.train_validate20.csv'), model=model)
dev_dataloader = DataLoader(dev_data, shuffle=False, batch_size=train_batch_size)
evaluator = EmbeddingSimilarityEvaluator(dev_dataloader)


2019-11-28 11:44:04 - Read STSbenchmark train dataset


Convert dataset:  31%|███       | 102/331 [00:00<00:00, 1015.12it/s]

2019-11-28 11:44:05 - Num sentences: 992
2019-11-28 11:44:05 - Sentences 0 longer than max_seqence_length: 0
2019-11-28 11:44:05 - Sentences 1 longer than max_seqence_length: 0
2019-11-28 11:44:05 - Read STSbenchmark dev dataset


Convert dataset: 100%|██████████| 331/331 [00:00<00:00, 1037.54it/s]

2019-11-28 11:44:05 - Num sentences: 331
2019-11-28 11:44:05 - Sentences 0 longer than max_seqence_length: 0
2019-11-28 11:44:05 - Sentences 1 longer than max_seqence_length: 0


In [0]:
# Configure the training. We skip evaluation in this example
warmup_steps = math.ceil(len(train_data)*num_epochs/train_batch_size*0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))


2019-11-28 11:44:05 - Warmup-steps: 25


In [0]:
# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path)

Convert Evaluating:   0%|          | 0/21 [00:00<?, ?it/s]

2019-11-28 12:13:06 - Evaluation the model on  dataset after epoch 0:



Epoch:   0%|          | 0/4 [31:53<?, ?it/s]

2019-11-28 12:15:59 - Cosine-Similarity :	Pearson: 0.7236	Spearman: 0.7283
2019-11-28 12:15:59 - Manhattan-Distance:	Pearson: 0.7016	Spearman: 0.7206
2019-11-28 12:15:59 - Euclidean-Distance:	Pearson: 0.6999	Spearman: 0.7206
2019-11-28 12:15:59 - Dot-Product-Similarity:	Pearson: 0.7218	Spearman: 0.7229
2019-11-28 12:15:59 - Save model to output/training_n2c2_sts_bert-2019-11-28_11-43-46


Epoch:  25%|██▌       | 1/4 [31:55<1:35:45, 1915.26s/it]

Iteration:   0%|          | 0/62 [00:00<?, ?it/s]

...


Convert Evaluating:   0%|          | 0/21 [00:00<?, ?it/s]

2019-11-28 13:50:59 - Evaluation the model on  dataset after epoch 3:



Epoch:  75%|███████▌  | 3/4 [2:09:49<32:20, 1940.76s/it]

2019-11-28 13:53:54 - Cosine-Similarity :	Pearson: 0.7893	Spearman: 0.7902
2019-11-28 13:53:55 - Manhattan-Distance:	Pearson: 0.7852	Spearman: 0.7913
2019-11-28 13:53:55 - Euclidean-Distance:	Pearson: 0.7820	Spearman: 0.7894
2019-11-28 13:53:55 - Dot-Product-Similarity:	Pearson: 0.7703	Spearman: 0.7678
2019-11-28 13:53:55 - Save model to output/training_n2c2_sts_bert-2019-11-28_11-43-46


Epoch: 100%|██████████| 4/4 [2:09:50<00:00, 1943.14s/it]


In [0]:
##############################################################################
#
# Load the stored model and evaluate its performance on STS benchmark dataset
#
##############################################################################

model = SentenceTransformer(model_save_path)
test_data = SentencesDataset(examples=sts_reader.get_examples("clinicalSTS2019.train_test20.csv"), model=model)
test_dataloader = DataLoader(test_data, shuffle=False, batch_size=train_batch_size)
evaluator = EmbeddingSimilarityEvaluator(test_dataloader)
model.evaluate(evaluator)

2019-11-28 13:53:56 - Load pretrained SentenceTransformer: output/training_n2c2_sts_bert-2019-11-28_11-43-46
2019-11-28 13:53:56 - Load SentenceTransformer from folder: output/training_n2c2_sts_bert-2019-11-28_11-43-46
2019-11-28 13:53:56 - loading configuration file output/training_n2c2_sts_bert-2019-11-28_11-43-46/0_BERT/config.json
2019-11-28 13:53:56 - Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "torchscript": false,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

2019-11-28 13:53:56 - loading weights file output/training_n2c2_sts_bert-2019-11-28_11-43-46/0_BERT/pytorch_model.bin


Convert dataset:  34%|███▎      | 111/331 [00:00<00:00, 1091.97it/s]

2019-11-28 13:53:58 - Model name 'output/training_n2c2_sts_bert-2019-11-28_11-43-46/0_BERT' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc). Assuming 'output/training_n2c2_sts_bert-2019-11-28_11-43-46/0_BERT' is a path or url to a directory containing tokenizer files.
2019-11-28 13:53:58 - loading file output/training_n2c2_sts_bert-2019-11-28_11-43-46/0_BERT/vocab.txt
2019-11-28 13:53:58 - loading file output/training_n2c2_sts_bert-2019-11-28_11-43-46/0_BERT/added_tokens.json
2019-11-28 13:53:58 - loading file output/training_n2c2_sts_bert-2019-11-28_11-43-46/0_BERT/special_tokens_map.json
2019-11-28 13:53:

Convert Evaluating:   0%|          | 0/21 [00:00<?, ?it/s]

2019-11-28 13:53:58 - Num sentences: 331
2019-11-28 13:53:58 - Sentences 0 longer than max_seqence_length: 0
2019-11-28 13:53:58 - Sentences 1 longer than max_seqence_length: 0
2019-11-28 13:53:58 - Evaluation the model on  dataset:


Convert Evaluating: 100%|██████████| 21/21 [02:32<00:00,  5.34s/it]

2019-11-28 13:56:31 - Cosine-Similarity :	Pearson: 0.7544	Spearman: 0.7408
2019-11-28 13:56:31 - Manhattan-Distance:	Pearson: 0.7258	Spearman: 0.7316
2019-11-28 13:56:31 - Euclidean-Distance:	Pearson: 0.7246	Spearman: 0.7336
2019-11-28 13:56:31 - Dot-Product-Similarity:	Pearson: 0.7424	Spearman: 0.7278


0.7408469748794851

In [0]:
!zip -r output.zip output

  adding: output/ (stored 0%)
  adding: output/training_n2c2_sts_bert-2019-11-28_11-43-46/ (stored 0%)
  adding: output/training_n2c2_sts_bert-2019-11-28_11-43-46/config.json (stored 0%)
  adding: output/training_n2c2_sts_bert-2019-11-28_11-43-46/1_Pooling/ (stored 0%)
  adding: output/training_n2c2_sts_bert-2019-11-28_11-43-46/1_Pooling/config.json (deflated 47%)
  adding: output/training_n2c2_sts_bert-2019-11-28_11-43-46/similarity_evaluation_results.csv (deflated 50%)
  adding: output/training_n2c2_sts_bert-2019-11-28_11-43-46/modules.json (deflated 51%)
  adding: output/training_n2c2_sts_bert-2019-11-28_11-43-46/0_BERT/ (stored 0%)
  adding: output/training_n2c2_sts_bert-2019-11-28_11-43-46/0_BERT/config.json (deflated 51%)
  adding: output/training_n2c2_sts_bert-2019-11-28_11-43-46/0_BERT/vocab.txt (deflated 53%)
  adding: output/training_n2c2_sts_bert-2019-11-28_11-43-46/0_BERT/pytorch_model.bin (deflated 7%)
  adding: output/training_n2c2_sts_bert-2019-11-28_11-43-46/0_BERT/spec

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!cp output.zip '/content/gdrive/My Drive/bert_finetuned_sts_biomedical.zip'